# FATES parameter ensemble evaluation 

This code evaluates the performance of 287 parameter ensemble member simulations against observations at a tropical forest test site, Barro Colorado Island, Panama. Parameter ensemble member simulations are identical (same model structure, initial conditions, and meteorological forcing) except that they differ in 12 plant trait parameters. Values for these plant trait parameters were sampled from observationally constrained distributions when possible.


# Load libraries

In [1]:
import netCDF4 as nc4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dateutil.parser
import datetime
import scipy
from scipy import stats

# Specifiy user defined parameters

In [2]:
# Number of years of model output to analyze
nyrs = 50

# Model output
# Filepath
model_filepath = 'data/'
# Model filenames where {1} = model_casenames, {2} = varfiletype
model_filenames =[
    'fates_clm5_fullmodel_bci_parameter_ensemble_1pft_slaprofile_{}_v001.I2000Clm50FatesGs.Cdf9b02d-Fb178808.2018-07-27.h{}.ensemble.sofar.nc',
    'fates_clm5_fullmodel_bci_parameter_ensemble_1pft_slaprofile_{}_v001.I2000Clm50FatesGs.Cdf9b02d-Fb178808.2018-07-27.h{}.ensemble.sofar.nc']

# Case names
# We analyze results for two separate ensembles. One ensemble is run at 367ppm CO2, 
# the second at 400ppm CO2. These atmospheric CO2 concentrations are the approximate
# concentrations at the beginning and end of the time period over which the observations
# that we use in our benchmarking analysis were made.
model_casenames = ['367ppm', '400ppm']

# Variable list for model output
varlist = ['TLAI','AGB','BA','GPP','FLH','FSH']

# Variable file type
# Model output differs in structure by variable.
# varfiletype: 0 = monthly, no tree size structure; 1 = annual, tree size structured.
# Order of varfiletype corresponds to variables in varlist.
varfiletype = [0,1,1,0,0,0]

# Variable conversion factor (order corresponds to varlist)
varconv = [1, 1, 1, 86400*365, 1, 1]

# Variable units after applying conversion factor (order corresponds to varlist)
varunits = ['$m^2/m^2$','$kgC/m^2$','$m^2/ha$','$gC/m^2/yr$','$W/m^2$','$W/m^2$']

# Degradation level for the observational range (as fractions, not percent)
dg = np.array([0.10])

# Load and preprocess data

### Model output

In [3]:
# Create a multidimensional array to store model output data
# model_data = [case, variable, ensemble member, time series]
# case: 0 = ensemble run at 367ppm CO2; 1 = ensemble run at 400ppm CO2.
# variable: 
    # 0 = Leaf area index
    # 1 = Above ground biomass
    # 2 = Basal area
    # 3 = Gross primary productivity
    # 4 = Latent heat flux
    # 5 = Sensible heat flux
# ensemble member: 0-286 = specifies randomly generated parameter set
# timeseries: annual mean values for n years

# Determine number of ensemble members for each model case
nens = nc4.Dataset(model_filepath + model_filenames[0].format(model_casenames[0],varfiletype[0])).variables[varlist[0]].shape[0]

# Empty array to be filled
model_data = np.zeros([len(model_casenames), len(varlist), nens, nyrs])


# Define a function to calculate time series of annual means for model output
def annmeants(filepath,var,varfiletype,nyrs,conv_factor):
    ''' Calculate time series of annual means for a model output variable.
    :param filepath (str): file path to data file
    :param var (str): name of variable to call from filename
    :param nyrs (int, float): number of years to analyze
    :param conv_factor (float): conversion factor specific to variable specified by var
    :return: 2-D array containing annual mean time series [ensemble member, nyrs] 
    '''
    
    # If model output is stored as monthly average for all tree sizes,
    # need to calculate annual mean.   
    if varfiletype == 0:
        
        # Load monthly time series
        # For all cases except latent heat flux (FLH):
        if var != 'FLH':
            mthts_temp = nc4.Dataset(filepath).variables[var][:,:,0]
        
        # For the special case of latent heat flux:
        elif var == 'FLH':
            # Sum of three terms:
            mthts_temp = (nc4.Dataset(filepath).variables['FCTR'][:,:,0] 
                          + nc4.Dataset(filepath).variables['FGEV'][:,:,0] 
                          + nc4.Dataset(filepath).variables['FCEV'][:,:,0])
        
        
        # Calculate annual mean time series for nyrs and convert units if necessary
        annmeants = np.nanmean(np.reshape((mthts_temp[:,int(-1*nyrs*12):] * conv_factor),
                                          (mthts_temp.shape[0],-1,12)),axis=2)
        
    # Else if model output is stored as annual mean but structured by tree size,
    # need to sum across tree sizes.
    elif varfiletype == 1:
        # Calculate annual mean time series for entire ecosystem by summing across tree sizes
        annmeants = np.squeeze(np.nansum((
                        nc4.Dataset(filepath).variables[var + '_SCLS'][:,int(-1*nyrs):,:]),
                        axis=2))
    
    mthts_temp = None
    
    return annmeants


# Fill array with model output data
for c in range(len(model_casenames)):
    
    for v in range(len(varlist)):      
        
        # Specify filepath for this case and variable
        filepath = model_filepath + model_filenames[c].format(model_casenames[c],varfiletype[v])
                
        # Add data for this case and variable to model_data array
        model_data[c, v, :, :] = annmeants(filepath, varlist[v], varfiletype[v], nyrs, varconv[v])

        filepath = None


### Observations

#### Gross primary productivity, latent heat flux, and sensible heat fluxes

Calculated from fluxtower data at Barro Colorado Island, Panama. Data from:<br>

Koven, C. D., et al. Benchmarking and Parameter Sensitivity of Predictions of Ecophysiological and Vegetation Dynamics using the Functionally Assembled Terrestrial Ecosystem Simulator (FATES) at Barro Colorado Island, Panama. In prep.

In [ ]:
# Code in this section by Charles D. Koven, Lawrence Berkeley National Lab

# Load and preprocess observations for GPP, LH, SH
# This results in arrays with shape(6 years, 12 months) for:
# gpp_gcm2y = timeseries of monthly mean gpp (gC/m2/yr); averaged across all 30min time points for each month)
# LE_monthyear = timeseries of monthly mean latent heat flux(W/m2);
# H_monthyear = timeseries of monthly mean sensible heat flux (W/m2);

bci_fluxtower_datafilename = 'data/BCI_v3.1.csv'

bci_fluxtower_data = np.genfromtxt(bci_fluxtower_datafilename, dtype={'names': ('date','tair','RH','vpd','p_kpa','PPT','Rs','Rs_dn','Rl_dn','Rl_up','Rnet','LE','H','Par_tot','Par_diff','SWC','ubar','ustar','WD','gpp','FLAG'),'formats': ('S16','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4')}, delimiter=',', skip_header=2)

ntim = len(bci_fluxtower_data)

time_start = datetime.datetime(2010, 1, 1, 0, 00)

tdelt = np.ma.masked_all(ntim)
month = np.ma.masked_all(ntim, dtype=np.int)
year = np.ma.masked_all(ntim, dtype=np.int)
gpp = np.ma.masked_all(ntim)
LE = np.ma.masked_all(ntim)
H = np.ma.masked_all(ntim)

for i in range(ntim):
    ts = dateutil.parser.parse(bci_fluxtower_data[i][0])
    tdelt[i] = (ts - time_start).days + (ts - time_start).seconds / 86400.
    month[i] = ts.month
    year[i] = ts.year
    gpp[i] = bci_fluxtower_data[i][19]
    LE[i] = bci_fluxtower_data[i][11]
    H[i] = bci_fluxtower_data[i][12]

H_masked = np.ma.masked_invalid(H)

nyears = (year.max() - year.min()) + 1
nmonths = nyears * 12

gpp_monthly = np.ma.masked_all(nmonths)
gpp_monthyear = np.ma.masked_all([nyears,12])
LE_monthyear = np.ma.masked_all([nyears,12])
H_monthyear = np.ma.masked_all([nyears,12])

for i in range(nyears):
    for j in range(12):
        mask = (year[:] == (year.min() + i)) * (month[:] == (j + 1))
        index = i*12 + j
        # gpp_monthly2[index] = gpp[mask].mean()                                                                                                                                                                                                                              
        if mask.sum() > 0:
            gpp_monthly[index] = (gpp * mask).sum() / mask.sum()
            gpp_monthyear[i,j] = gpp_monthly[index]
            LE_monthyear[i,j] = (LE * mask).sum() / mask.sum()
        H_monthyear[i,j] = H_masked[mask].mean()

months = np.arange(12)

### change gpp units to grams or carbon per meter squared per year
gpp_gcm2y = gpp_monthyear * 1e-6 * 12.0107 * (86400 * 365.25)

In [ ]:
# Calculate annual mean values for gross primary productivity, latent heat flux, and sensible heat flux
# Results in obs_data_flux = annual means indexed by [startmonth (0 = July, 1= Sept), variable, year]
#    variable: 0 = gross primary productivity, 1 = latent heat flux, 2 = sensible heat flux
#    startmonth: index for annual mean calculations starting from different months
#        There are two time series for each variable because the observations span incomplete years.
#        To use all data available in our analysis, we calculate have one time series of means 
#        starting with July and the second starting with Sept (this uses all months available).
#    year: year in the annual mean time series

# Function to calculate annual mean time series from monthly data
def annmeants_fluxobs(mthts,startmth):
    ''' Calculate time series of annual means from fluxtower estimates.
    :param mthts (float): 2-D array containing fluxtower observations [years, months]
    :param startmth (int): number corresponding to start month for this annual mean time series
                            (e.g. 7 = start with July, 9 = start with Sept)
    :return: vector containing annual mean time series [nyrs] 
    '''
    # Discard number of months specified by dif
    mthts_dif = np.reshape(mthts,(1,-1))[:,startmth-1:startmth-1-12]
    # Calculate annual mean time series
    annmeants = np.nanmean(np.reshape(mthts_dif,(5,12)),axis=1)
    
    return annmeants

# Specify start months for observations
startmonth_list = np.array([7,9])

# Number of years
nyrs_obsflux = len(annmeants_fluxobs(gpp_gcm2y,startmonth_list[0]))

# Create empty array to fill with observation time series for GPP, LH, SH
obs_data_flux = np.zeros([len(startmonth_list), 3, nyrs_obsflux])

for x in range(len(startmonth_list)):
    obs_data_flux[x,0,:] = annmeants_fluxobs(GPP_monthyear,startmonth_list[x])
    obs_data_flux[x,1,:] = annmeants_fluxobs(LE_monthyear,startmonth_list[x])
    obs_data_flux[x,2,:] = annmeants_fluxobs(H_monthyear,startmonth_list[x])

#### Leaf area index observations

Data was captured using GraphClick software from Figure 7a of: <br>
Detto, M., Wright, S. J., Calderón, O., & Muller-Landau, H. C. (2018). Resource acquisition and reproductive strategies of tropical forest in response to the El Niño-Southern Oscillation. Nature Communications, 9(1), 913. https://doi.org/10.1038/s41467-018-03306-9

Note. Monthly data consists of spatial means across 188 locations taken approximately once a month within our test site (not temporal means).

In [4]:
# Return: obs_data_lai: 2D array containing annual mean leaf area index [sample number, years]
#    sample: 0 = sample months starting from January; 1 = starting from Sept
#    Note. As observations consist of 2 years and 8 months of data, we sample both
#    the first set of full years (starting in January) and last set of full years
#    (starting in September)

# File path
filepath = 'data/LAI_Detto2018Obs.csv'

# Monthly spatial means
lai_mthts = np.asarray([col[2] for col in (pd.read_csv(filepath)).values])

# Specify start months for observations
startmonth_list = np.array([1,9])

# Number of annual means per sample
nyears_lai = round(len(lai_mthts)/12-0.5)

# Empty array to be filled [sample number, years]
obs_data_lai = np.zeros([len(startmonth_list), nyears_lai])

# Calculate annual means (starting in January, then in September)
for x in range(len(startmonth_list)):
    obs_data_lai[x,:] = np.nanmean(np.reshape(lai_mthts[startmonth_list[x]-1:24+startmonth_list[x]-1],(nyears_lai,12)),1)

#### Above-ground biomass observations

Data from:

Meakem, V., Tepley, A. J., Gonzalez-Akre, E. B., Herrmann, V., Muller-Landau, H. C., Wright, S. J., et al. (2018). Role of tree size in moist tropical forest carbon cycling and water deficit responses. New Phytologist, 219, 947–958. https://doi.org/doi:10.1111/nph.14633
<br>
 
Baraloto, C., Molto, Q., Rabaud, S., Hérault, B., Valencia, R., Blanc, L., et al. (2013). Rapid simultaneous estimation of aboveground biomass and tree diversity across Neotropical forests: a comparison of field inventory methods. Biotropica, 45(3), 288–298. https://doi.org/10.1111/btp.12006 
<br>

Feeley, K. J., Davies, S. J., Ashton, P. S., Bunyavejchewin, S., Supardi, M. N., Kassim, A. R., et al. (2007). The role of gap phase processes in the biomass dynamics of tropical forests. Proceedings of the Royal Society of London B: Biological Sciences, 274(1627), 2857–2864. https://doi.org/10.1098/rspb.2007.0954
<br>

Note. We use the range across two different allometry (i.e. tree shape) calculations (standard and Chave) from Meakem et al. (2018) to bracket the overall uncertainty in biomass. Alternatively, we could estimate carbon biomass from biomass reported in Baraloto et al. (2013) and Feeley et al. (2007). This alternative method was tested an yeilds similar results.

In [ ]:
# Return: obs_data_agb: vector of above ground biomass (KgC/m2)
#    from estimates from Meakem et al. 2018 [allometry calculation] 
#   allometry calculation: 0 = standard, 1 = Chave

filepath = 'data/BCI_biomass.csv'

# Carbon above ground biomass from Meakem et al. 2018 (MgC/ha) 
cbiomass_obs_Mgha = np.asarray([col[2] for col in (pd.read_csv(filepath)).values])[-2:,]
# Convert from MgC/ha to KgC/m2
ha_to_m2 = 1/10000
Mg_to_kg = 1000
obs_data_agb = cbiomass_obs_Mgha * ha_to_m2 * Mg_to_kg

# Alternatively could use the following in our analysis by
# uncommenting the last line here (results are similar)
# Total aboveground biomass (Mg biomass/ha) from census data 
# (Baraloto et al. 2013, Biotropica; Feeley et al. 2007, Proc Roy Soc B)
# Years: 2005,1985,1990,1995,2000,2005
agb_biomass_obs = np.asarray([col[1] for col in (pd.read_csv(filepath)).values])[:-2,]
# Estimate of carbon biomass from biomass using scaler of 0.47 following Meakem et al. 2018
obs_data_agb_v2 = agb_biomass_obs*0.47
#obs_data_agb = obs_data_agb_v2

#### Basal area observations

Data from census surveys of a 50-hectare plot on Barro Colorado Island, Panama, from:

Condit, R. S., Aguilar, S., Perez, R., Lao, S., Hubbell, S. P., & Foster, R. B. (2017). Barro Colorado 50-ha Plot Taxonomy as of 2017. https://doi.org/10.25570/stri/10088/32990

Condit, R., Lao, S., Pérez, R., Dolins, S. B., Foster, R., & Hubbell, S. (2012). Barro Colorado forest census plot data (version 2012). Center for Tropical Forest Science Databases. Https://Dx. Doi. Org/10.5479/Data. Bci. https://doi.org/http://dx.doi.org/10.5479/data.bci.20130603

Condit, R. (1998). Tropical forest census plots. Berlin, Germany, and Georgetown, Texas: Springer-Verlag and R. G. Landes Company.

Hubbell, S. P., Foster, R. B., O'Brien, S. T., Harms, K. E., Condit, R., Wechsler, B., et al. (1999). Light-gap disturbances, recruitment limitation, and tree diversity in a neotropical forest. Science, 283(5401), 554–557. https://doi.org/10.1126/science.283.5401.554 

Note. Data from 1992, 1996, 2001, 2006, and 2011 are used herein. Previous census years are excluded for consistency as a different measurement method was used.

In [5]:
# Return: obs_data_ba: vector containing basal area (m^2/ha) by census year [years]

filepath = 'data/census_bmks_bci_171208.nc'

# Load basal area median values for the last 5 census dates
# Data stored in filepath as [census number, tree diameter size class, distribution percentiles (0.05,0.5,0.95)]
basalarea_bysize = nc4.Dataset(filepath).variables['basal_area_by_size_census'][-5:,:,1]
# Calculate total median value for site by summing across tree size classes
obs_data_ba = np.nansum(basalarea_bysize,1)

# Evaluate performance of each parameter ensemble member against observations

### Metric #1: Error Rate

The error rate measures the percent of model annual means that fall within observed range for each variable and ensemble member.<br>

The observed range is defined as the minimum and maximum across observations. To account for relatively small sample sizes and potential measurment error within the observations we extend the observational range by the 10% in either direction.

In [ ]:
# Return:  error_rate_array: a 4-D array containing error rates
#    [case, variable, model ensemble member, degradation level]

# Specify names of observed data arrays in order corresponding to varlist
obs_data_list = [obs_data_lai,obs_data_agb,obs_data_ba,
                 obs_data_flux[:,0,:],obs_data_flux[:,1,:],
                 obs_data_flux[:,2,:]]


#Function to calculate error rate 
def error_rate(model_ts,obs_ts,dg):
    '''Function calculates the error rate, meaning the percentage of
    model annual means that fall within the observed range for each model
    ensemble member
    param model_ts: a 2-D array containing the time series of annual means
       for a given variable for all model ensemble members [ensemble member, years]
    param obs_ts: an n-D array containing the observed time series for
        the given variable
    param dg: a vector of any length specifying the levels of range degradation
        to test
    return error_rate: a 2-D array containing the error rates for each 
        ensemble member [ensemble member, degradation rates (dg)]'''
    
    # Number of ensemble members
    nens = model_ts.shape[0]
    
    # Empty array to fill
    error_rate = np.zeros([nens])

    # Observed minimum and maximum
    obs_min = np.nanmin(obs_ts)
    obs_max = np.nanmax(obs_ts)
    
    error_rate = 100*np.nansum(np.where((model_ts <= obs_min*(1-dg)) | (model_ts >= obs_max*(1+dg)),1,0),1)/model_ts.shape[1]
    return error_rate


# Calculate error rate for all cases and variables
error_rate_array = np.zeros([len(model_casenames),len(varlist), nens])
for i in range(len(model_casenames)):
        for j in range(len(varlist)):
            error_rate_array[i,j,:] = error_rate(model_data[i,j,:,:], obs_data_list[j], dg)     

### Metric #2:  Normalized root mean square error (NRMSE)

The normalized root mean square error (NRMSE) measures the distance between each ensemble member's model output from the observed mean and normalizes this distance by the observed range. The normalized root mean square error is calculated for each variable and ensemble member.

In [ ]:
# Calculate Normalized Root Mean Square Error (RMSE/Observed Range)
# Returns nrsmse_array a 3-D array containing NRMSE for each 
#[case, variable, ensemble member]

# Function to calculate normalized root mean square errer (nrmse)
# When multiple observation time series are available, this function
# calculates the nrmse for each time series and then selects the 
# lowest of those nrmse values.
def nrmse(model_ts,obs_ts):
    '''Function calculates the normalized root mean square error for each model
    ensemble member
    param model_ts: a 2-D array containing the time series of annual means
       for a given variable for all model ensemble members [ensemble member, years]
    param obs_ts: an n-D array containing the observed time series for
        the given variable
    return nrmse: a vector containing the normalized root mean square error 
        for each ensemble member [ensemble member]'''
    
    # Number of ensemble members
    nens = model_ts.shape[0]

    # If multiple observation time series, 
    # take the lowest NRMSE for each ensemble member
    try:
        if obs_ts.shape[1]>0:
            # Number of observation time series
            nobs = obs_ts.shape[0]
            obs_min = np.nanmin(obs_ts,axis=1)
            obs_max = np.nanmax(obs_ts, axis=1)
            obs_mean = np.nanmean(obs_ts,axis=1)
            
            temp_nrmse = np.zeros([nobs,nens])
            
            for obsnum in range(nobs):
                temp_nrmse[obsnum,:] = np.sqrt(np.nansum((obs_mean[obsnum] - model_ts[:,:])**2,axis=1) / model_ts.shape[1]) / (obs_max[obsnum]-obs_min[obsnum])
                
            nrmse = np.nanmin(temp_nrmse,axis=0)

            temp_nrmse = None
        
    # Otherwise, simply calculate NRMSE
    except IndexError:
        obs_min = np.nanmin(obs_ts,axis=0)
        obs_max = np.nanmax(obs_ts,axis=0)
        obs_mean = np.nanmean(obs_ts,axis=0)    
        
        nrmse = np.sqrt(np.nansum((obs_mean - model_ts[:,:])**2,axis=1) / model_ts.shape[1]) / (obs_max-obs_min)

    return nrmse
    

# Calculate NRMSE for all cases and variables
nrmse_array = np.zeros([len(model_casenames),len(varlist), nens])

for i in range(len(model_casenames)):
        for j in range(len(varlist)):
            nrmse_array[i,j,:] = nrmse(model_data[i,j,:,:], obs_data_list[j])

### Weighted average across variables

We calculate weighted averages across variables for both the normalized root mean square error and the error rate. We calculate and consider three different weighting approaches to ensure that our selection of high-performing parameter sets is robust to weighting method. These three weighting approaches we use are:

1. Even:  All variables are evenly weighted.

2. Structure:  This weighting favors structural ecosystem properties (leaf area index, above-ground biomass, and basal area). This weighting scheme reflects the likelihood that structural property measurements at our test site include less uncertainty than flux measurements.

3. Correlation: This weighting scheme is informed by correlations between individual variable scores. Ensemble member performance in flux variables (gross primary productivity, sensible heat, and latent heat) was correlated with leaf area index and with one another. As leaf area index observations likely include smaller measurement uncertainty, we chose to weight leaf area index more highly at the expense of flux observations. We also reduced the weightings of basal area and above-ground biomass to account for their correlation with one another.

#### Error Rate

In [ ]:
# Even weighting across all variables
er_wavg_even = np.nansum(error_rate_array,1) / error_rate_array.shape[1]


# Weighted average favoring structural properties:
# leaf area index, aboveground biomass, basal area
w = 0.3
er_wavg_strct = ( w*(error_rate_array[:,0,:]) 
            + w*(error_rate_array[:,1,:])
            + w*(error_rate_array[:,2,:])
            + (1-3*w)*((error_rate_array[:,3,:])
                +(error_rate_array[:,4,:])
                +(error_rate_array[:,5,:]))/3)

# Alternative weighting based on correlations between error rates across variables
# 1. Weighted average giving greatest weight to LAI, then to combination of AGB and BA (which are correlated with one another),
# and small weight to fluxes which are correlated with one another and with LAI.
w1 = 0.4
w2 = 0.25
w3 = 0.1
er_wavg_corr = ( w1*(error_rate_array[:,0,:])  
            + w2*(error_rate_array[:,1,:])  
            + w2*(error_rate_array[:,2,:]) 
            + w3*((error_rate_array[:,3,:])
                +(error_rate_array[:,4,:])
                +(error_rate_array[:,5,:]))/3)

# Note. Code for er_wavg_strct and er_wavg_corr should be updated 
# to allow the flexibility to add additional variables

#### NRMSE

In [ ]:
# Calculate weighted average Distance scores
# Here we use the weighted Euclidean Distance to calculate the distance of model output from the mean observations in multivariate space:
#  Square each variable score before weighting, then take squareroot of the weighted sum;
#  Use the time period that fits model output best (e.g. jan vs sept start)


# Weighted average with even weighting across all variables
w = 1/6
nrmse_wavg_even = np.sqrt(w*(nrmse_array[:,0,:])**2 
                                  + w*(nrmse_array[:,1,:])**2 
                                  + w*(nrmse_array[:,2,:])**2 
                                  + w*(nrmse_array[:,3,:])**2 
                                  + w*(nrmse_array[:,4,:])**2 
                                  + w*(nrmse_array[:,5,:])**2)


# Define weighted average favoring LAI, AGB, and BA (our structural properties)
w = 0.3
nrmse_wavg_strct = np.sqrt(w*(nrmse_array[:,0,:])**2 
                        + w*(nrmse_array[:,1,:])**2 
                        + w*(nrmse_array[:,2,:])**2 
                        + (1-3*w)*((nrmse_array[:,3,:])**2 
                                    +(nrmse_array[:,4,:])**2 
                                    +(nrmse_array[:,5,:])**2)/3)

# Weighted average giving greatest weight to LAI, then to combination of AGB and BA (which are correlated with one another),
# and small weight to fluxes which are correlated with one another and with LAI.
w1 = 0.4
w2 = 0.25
w3 = 0.1
nrmse_wavg_corr = np.sqrt(w1*(nrmse_array[:,0,:])**2 
                          + w2*(nrmse_array[:,1,:])**2 
                          + w2*(nrmse_array[:,2,:])**2 
                          + w3*((nrmse_array[:,3,:])**2 
                                  +(nrmse_array[:,4,:])**2 
                                  +(nrmse_array[:,5,:])**2)/3)

# Note. The above code should be updated 
# to allow the flexibility to add additional variables

# Rank ensemble members by performance
Here we assign an overall rank to each ensemble member based on its performance across both performance metrics (error rate and NRMSE), a three weighting schemes (even, structure, and correlated), and two cases (low and high atmospheric carbon dioxide concentration). The goal of this analysis is to identify parameter ensemble members (and thus parameter sets) that robsutly perform well at our test site.

In [ ]:
all_avg_array = np.stack([er_wavg_even,nrmse_wavg_even,
                             er_wavg_strct,nrmse_wavg_strct,
                             er_wavg_corr,nrmse_wavg_corr])

rank_array = scipy.stats.mstats.rankdata(all_avg_array,axis=2)

# Sum ranks across cases and ranking methods
sum_rank_array = np.nansum(np.nansum(rank_array,axis=0),axis=0)

# Sort the index number for each ensemble member by their summed rank (best to worst performance)
sum_rank_index = np.argsort(sum_rank_array)

#Print Index # for Top 10 Ensemble Members
highperform_num = np.transpose(sum_rank_index)[:10,]+1
highperform_indx = np.transpose(sum_rank_index)[:10,]
print("Highest Performing Ensemble Member Numbers: ", highperform_num)

# Plot performance metrics for high-performing and all ensemble members

In [ ]:
# param dg: specifies degradation level of observational range
# return error_heatdata: error rates by [case, variable, ensemble member]
# return nrmse_heatdata: NRMSE by [case, variable, ensemble member]
#    where variable index 0-5 = variables in order of varlist, 
#                         6-8 = weighted average using even, structure, and
#                               correlated weighted methods respectively

# Concatenate data for error rate heatmap
error_rate_wavg_array = np.stack([er_wavg_even,er_wavg_strct,er_wavg_corr],axis=1)
error_heatdata = np.concatenate([error_rate_array,error_rate_wavg_array],axis=1)

# Concatenate data for NRMSE heatmap
nrmse_wavg_array = np.stack([nrmse_wavg_even,nrmse_wavg_strct,nrmse_wavg_corr],axis=1)
nrmse_heatdata = np.concatenate([nrmse_array,nrmse_wavg_array],axis=1)

# Metric/Variable labels
heat_var_labels = ["LAI","AGB","BA",
                "GPP","LH","SH","Av$_{E}$","Av$_{S}$","Av$_{C}$"]

# Ensemble member labels
# For 10 highest performing ensemble members
ens10 = [str(int(x)) for x in highperform_num]
# For all ensemble members (label every 25th ensemble member)
ens = np.array(range(25,300,25))
enslist = [str(x) for x in ens]

In [ ]:
def heatsubplotfxn(heatdata, casenum, minval, maxval, plotnum, metriclabel, highperform_indx, heat_var_labels, ens10):

    #Subplot indexing paramter
    i =2
    
    # Highest Performing Ensemble Members
    ax1 = plt.subplot(3,i,plotnum)
    im1 = ax1.imshow(heatdata[casenum,:,highperform_indx],vmin = minval, vmax = maxval,cmap="viridis_r",aspect='auto')

    ax1.set_xticks(np.arange(len(heat_var_labels)))
    ax1.xaxis.tick_top()
    ax1.set_xticklabels(heat_var_labels)
    ax1.xaxis.set_label_position('top')

    ax1.set_ylabel('High Performing Parameter Sets (#)')
    ax1.set_yticks(np.arange(len(ens10)))
    ax1.set_yticklabels(ens10)

    # All Ensemble Members
    ax2 = plt.subplot(3,i,(i+plotnum,i*2+plotnum))
    im2 = ax2.imshow(np.transpose(heatdata[casenum,:,:]),vmin = minval, vmax = maxval,cmap="viridis_r",aspect='auto')
    # Create colorbar
    cbar = ax1.figure.colorbar(im2, ax=ax2, orientation="horizontal", pad=0.025)
    cbar.ax.set_xlabel(metriclabel, fontsize = 16, fontweight ='bold')
    ax2.set_xticks([]) # to hide xticks/labels

    ax2.set_ylabel('All Parameter Sets (#)')
    ax2.set_yticks(ens)
    ax2.set_yticklabels(enslist)

#### Case #1: Simulations run with approximate carbon dioxide concentration at the beginning of observational period (367 ppm CO2)

In [ ]:
fig1 = plt.figure(figsize=(12,12))

# Set case to 367ppm CO2
casenum = 0

# Plot Error Rate
plotnum = 1
heatsubplotfxn(error_heatdata, casenum, 0, 100, plotnum, 'A.  Error Rate (%)', 
               highperform_indx, heat_var_labels, ens10)

# Plot NRMSE
plotnum = plotnum+1
heatsubplotfxn(nrmse_heatdata, casenum, 0, 10, plotnum, 'B.  NRMSE', 
               highperform_indx, heat_var_labels, ens10)

plt.tight_layout()

#### Case #2: Simulations run with approximate carbon dioxide concentration at the end of observational period (400 ppm CO2)

In [ ]:
fig2 = plt.figure(figsize=(12,12))

# Set case to 400ppm CO2
casenum = 1

# Plot Error Rate
plotnum = 1
heatsubplotfxn(error_heatdata, casenum, 0, 100, plotnum, 'A.  Error Rate (%)',
              highperform_indx, heat_var_labels, ens10)

# Plot NRMSE
plotnum = plotnum+1
heatsubplotfxn(nrmse_heatdata, casenum, 0, 10, plotnum, 'B.  NRMSE',
              highperform_indx, heat_var_labels, ens10)

plt.tight_layout()